**Môn học: Machine Learning**

**Giảng viên: **TS. Lê Thành Sách

**Bài tập lớn số 1**
: Bản hiện thực mạng CNN với cupy

*Sinh viên:*

*Nguyễn Hoàng Lộc - MSSV: 1511849*

*Nguyễn Hữu Anh Tiến - MSSV: 1513448 *

# Install, import & setup

In [0]:
!apt -y install libcusparse8.0 libnvrtc8.0 libnvtoolsext1
!ln -snf /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so.8.0 /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so
!pip install https://github.com/kmaehashi/chainer-colab/releases/download/2018-02-06/cupy_cuda80-4.0.0b3-cp36-cp36m-linux_x86_64.whl
!pip install 'chainer==4.0.0b3'
!curl https://colab.chainer.org/install | sh -

In [2]:
import numpy 
import cupy as cp
import math
import numpy 
import struct
import sys
import time
import keras
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
eps = cp.finfo(float).eps
from google.colab import drive

Using TensorFlow backend.


In [3]:
drive.mount('/content/drive')
sys.path.append('drive/My Drive/Colab Notebooks/') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load data, format & visualize

In [0]:
train_images_path = 'drive/My Drive/Colab Notebooks/Asgmt_ML_CNN/data/train-images'
train_labels_path = 'drive/My Drive/Colab Notebooks/Asgmt_ML_CNN/data/train-labels'
test_images_path = 'drive/My Drive/Colab Notebooks/Asgmt_ML_CNN/data/t10k-images'
test_label_path = 'drive/My Drive/Colab Notebooks/Asgmt_ML_CNN/data/t10k-labels'


def read_idx(filename):
    with open(filename, 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))       
        return numpy.fromstring(f.read(), dtype=cp.uint8).reshape(shape)

def load_mnist_data():
  train_data = [read_idx(train_images_path), read_idx(train_labels_path)]
  test_data = [read_idx(test_images_path), read_idx(test_label_path)]
  return train_data, test_data

In [5]:
train_data, test_data = load_mnist_data()
X_train = train_data[0]
y_train = train_data[1]
X_test = test_data[0]
y_test = test_data[1]

X_train = cp.array(X_train)
y_train = cp.array(y_train)
X_test = cp.array(X_test)
y_test = cp.array(y_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  # This is added back by InteractiveShellApp.init_path()


In [0]:
# fig = plt.figure()
# for i in range(9):
#   plt.subplot(3,3,i+1)
#   plt.tight_layout()
#   plt.imshow(X_train[i], cmap='gray', interpolation='none')
#   plt.title("Digit: {}".format(y_train[i]))
#   plt.xticks([])
#   plt.yticks([])

In [7]:
img_rows = 28
img_cols = 28
#Chuyển ảnh về format channel first
X_train_norm = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test_norm = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
input_shape = (1, img_rows, img_cols)

#Chuyển ảnh về dạng channel last
# X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
# X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
# input_shape = (img_rows, img_cols, 1)

X_train_norm = X_train_norm.astype('float32')
X_test_norm = X_test_norm.astype('float32')
X_train_norm /= 255
X_test_norm /= 255

# X_train_crop = cp.array([x[0, 12:16, 12:16] for x in X_train_norm])
# X_test_crop = cp.array([x[0, 12:16, 12:16] for x in X_test_norm])
# X_train_crop = X_train_crop.reshape(60000, 1, 4, 4)
#X_test_crop = X_test_crop.reshape(X_test.shape[0], 1, img_rows, img_cols)

print('X_train shape:', X_train.shape)

X_train shape: (60000, 28, 28)


In [8]:
#Chuyển tập target về dạng one-hot
def get_one_hot(targets, nb_classes):
    res = cp.eye(nb_classes)[cp.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])
  
num_category = 10 #ứng với 10 label từ 0->9
y_train_norm = get_one_hot(y_train, num_category)
y_test_norm = get_one_hot(y_test, num_category)
print('y_train shape:', y_train_norm.shape)

y_train shape: (60000, 10)


# --------------------CNN - numpy inplementation--------------------

#Model - Sequential

In [0]:
#Class đóng vai trò là model, quản lí việc forward và backward cho các layers
class Sequential: 
  def __init__(self):
    self.layer_list = list()
    
  def add(self, new_layer):
    self.layer_list.append(new_layer)
    
  def compile(self, loss = 'cross_entropy'):
    if loss == 'cross_entropy':
      self.loss = CrossEntropy()
    else:
      pass #Có thể hiện thực thêm nhiều hàm lỗi khác
  
  def fit(self, X_train, y_train, batch_size = 128, num_epoch = 10, validation_data = None):
    self.X_train = X_train
    self.y_train = y_train
    
    train_loss_list = list()
    val_loss_list = list()
    epoch_time_list = list()

    for epoch in range(num_epoch):
      epoch_loss = list()
      for num_batch in range(X_train.shape[0] // batch_size):
        current_index = num_batch*batch_size    
        image_batch = X_train[current_index : current_index + batch_size] #Lấy các ảnh ứng với epoch
        target_batch = y_train[current_index : current_index + batch_size] #Lấy các target ứng với các ảnh

        for index, image in enumerate(image_batch):
          output = self.feed_forward(image) #Feed từng ảnh qua mạng
          self.loss.feed_forward(output, target_batch[index]) #Add ouput của layer cuối cùng vào hàm loss

        batch_avg_loss = self.loss.get_avg_loss(batch_size)
        epoch_loss.append(batch_avg_loss)
#           avg_grad = self.loss.backward(batch_size)
        avg_grad = target_batch[index]
        self.backward(avg_grad)
        self.loss.reset()
      
      epoch_avg_loss = sum(epoch_loss) / len(epoch_loss)

      train_loss_list.append(epoch_avg_loss)
      epoch_time_list.append(epoch)
      
      if validation_data is not None:
        val_loss = self.evaluate(validation_data[0], validation_data[1])
        val_loss_list.append(val_loss)
        print("Epoch ", epoch,"- Training loss: ", cp.round(epoch_avg_loss, 14), " - Validating loss: ", val_loss)
      else:
        print("Epoch ", epoch,"- Training Loss: ", epoch_avg_loss)   
          
#     plt.plot(epoch_time_list, train_loss_list)
#     plt.plot(epoch_time_list, val_loss_list)
#     plt.legend(('Train', 'Val'))
#     plt.xlabel('Epoch:')
#     plt.ylabel('Loss: ')
#     plt.show(block=False)
       
           
  def feed_forward(self, data, isTrain = True):
    layer_input = data
    for layer in self.layer_list:
      print('Feedforward through layer: ',type(layer)) ##
      if not isTrain and type(layer) is Dropout: #Biến isTrain để bypass layer Dropout
        continue
      layer_input = layer.feed_forward(layer_input)
#       print("Output :", cp.around(layer_input, decimals = 4), layer_input.shape) ##

    #Sau khi feed qua tất cả layer thì biến layer_input chứa output của layer cuối cùng
    return layer_input 
  
  def backward(self, dY, isTrain = True):
    layer_grad = dY
    for layer in reversed(self.layer_list):
      print('Back-forward through layer: ', type(layer)) ##
      if isTrain == False and type(layer) is Dropout:
        continue
      layer_grad = layer.backward(layer_grad)
#       print ("Output: ", cp.around(layer_grad, decimals = 4), layer_grad.shape) ##
      
    
  def evaluate(self, X_test, y_test):
    output_list = list()
    for index, image in enumerate(X_test):
      output = self.feed_forward(image, isTrain = False) 
      output_list.append(output)
      self.loss.feed_forward(output, y_test[index])
    avg_loss = self.loss.get_avg_loss(len(y_test))
    self.loss.reset()
    return avg_loss
  
  def predict(self, X_test):
    output_list = list()
    for index, image in enumerate(X_test):
      output = self.feed_forward(image, isTrain = False) 
      output_onehot = cp.zeros_like(output)
      output_onehot[output.argmax()] = 1
      output_list.append(output_onehot)
    return output_list

  def predict_class(self, X_test):
    output_list = self.predict(X_test)
    output_list_class = list()
    for one_hot in output_list:
      num_class,  = cp.where(one_hot== 1)[0]
      output_list_class.append(num_class)
    return output_list_class

# Activation 




## ReLu

In [0]:
class ReLU:
  def __init__(self):
    self.data = None 
    
  def feed_forward(self, data):
    data[data<0] = 0
    data = data/ (cp.amax(data) + eps)
    self.data = data
    return data
  
  def backward(self, dY):
    max_data = cp.amax(self.data)
    self.data[self.data>0] = 1
    return cp.multiply(self.data, dY)*max_data

## Softmax



In [0]:
class Softmax():
  def __init__(self):
    self.cache = None
  
  def feed_forward(self, X):
    self.cache = cp.exp(X) / cp.sum(cp.exp(X), axis = 0)
    return self.cache
  
  def backward(self, dY):
    return self.cache - dY

# Loss

In [0]:
class CrossEntropy():
  def __init__(self):
    self.reset()
  
  def reset(self): #Gọi sau mỗi batch
    self.gradient = None #Chứa tổng gradient để backward cho mỗi batch
    self.total_loss = None #Chứa tổng loss của mỗi batch
    
  def feed_forward(self, output, target):
    temp = -target*(1/(cp.log(2)*output))
    if self.gradient is None:
      self.gradient = temp
    else:
      self.gradient += temp 
      
    loss = cp.sum(-1*target * cp.log2(output + eps))
    if self.total_loss is None:
      self.total_loss = loss
    else: 
      self.total_loss += loss
      
  def get_avg_loss(self, batch_size):
    return self.total_loss / batch_size
  
  def backward(self, batch_size):
    return self.gradient/batch_size

#Layers

## Convolution2D

In [0]:
class Convolution2D: 
  def __init__(self, num_filter, filter_size, input_shape, padding = 0, stride = 1, init_weights = None, init_bias = None, activation = 'relu', learning_rate = 0.01):
    self.num_filter = num_filter
    self.filter_size = filter_size
    self.input_shape = input_shape
    self.weights = init_weights
#     self.bias = init_bias
    self.padding = padding
    self.stride = stride
    if activation == 'relu':
      self.activation = ReLU()
    else:
      pass #Có thể hiện thực nhiều hàm activation khác
    self.lrn_rate = learning_rate
      
    if self.weights is None:
      rand_number = math.sqrt(6/(input_shape[0]+num_filter))
      self.weights = cp.random.uniform(low = -rand_number, high = rand_number, size = (num_filter, input_shape[0] ,filter_size[0], filter_size[1]))#Channel first format: param 2 -> số channel input 
#     if self.bias is None:
#       self.bias = cp.zeros(num_filter)

      
  def set_input(self, data): #Padding ảnh input với 0
    self.input = cp.zeros((self.input_shape[0], self.input_shape[1] + self.padding*2, self.input_shape[2] + self.padding*2))
    for channel in range(self.input_shape[0]):
        self.input[channel] = cp.pad(data[channel], pad_width = self.padding, mode = 'constant', constant_values = 0)
  
  def feed_forward(self, data):
    self.set_input(data)
    self.convert_X_conv()
    self.convert_W_conv()
    self.convert_W_flat()
    self.output = None
#     print ("Weights of Convolution forward", self.weights, self.weights.shape) ##
    for filter in range(self.num_filter):
      test = cp.matmul(self.W_flat_list[filter], self.X_conv)
      if self.output is None:
        self.output = test
      else:
        self.output = cp.concatenate((self.output, test), axis = 0)
    return self.activation.feed_forward(self.output.reshape((self.num_filter, self.o1, self.o2)))

  def backward(self, dY_flat):
    dY_flat = self.activation.backward(dY_flat)
    dX = cp.zeros((self.input_shape[0], self.input.shape[1], self.input.shape[2]))
    for filter in range(self.num_filter):    
      dY_flatten = dY_flat[filter, :, :].flatten()
      #print(dY_flatten.shape)    
      dW = cp.matmul(self.X_conv, dY_flatten).reshape(self.input_shape[0], self.filter_size[0], self.filter_size[1])      
      dX += cp.matmul(dY_flatten.reshape(1, dY_flatten.shape[0]) ,self.W_conv_list[filter]).reshape((self.input_shape[0], self.input.shape[1], self.input.shape[2]))           
      self.weights[filter] -= dW*self.lrn_rate
#     print ("Weights of Convolution backward", self.weights, self.weights.shape)##
    return dX
    
    
  def convert_W_flat(self):
    self.W_flat_list = list()
    for filter in range(self.num_filter):
      W_flat = None
      for channel in range(self.input_shape[0]):
        temp = self.weights[filter, channel, :, :].flatten()
        if W_flat is None:
          W_flat = temp
        else:
          W_flat = cp.concatenate((W_flat, temp), axis = 0)
      self.W_flat_list.append(W_flat)

    
  def convert_X_conv(self):
    #im2col cho các channel của input
    self.X_conv = None
    for channel in range(self.input_shape[0]):
      temp = self.im2col(self.input[channel])
      if self.X_conv is None:
        self.X_conv = temp
      else:
        self.X_conv = cp.concatenate((self.X_conv, temp), axis=0) #Mở rộng chiều rows
  
  
  def convert_W_conv(self):
    self.W_conv_list = cp.zeros((self.num_filter, self.o1*self.o2, self.input.shape[1]*self.input.shape[2]*self.input_shape[0]))
    #kernel2col cho tất cả các kernel (tất cả channel)
#     print("-1")
    for filter in range(self.num_filter):
#       print("Filter :",filter)
      W_conv = None
      for channel in range(self.input_shape[0]):
#         print("\t\tChannel :",channel)
        temp = self.kernel2row(self.weights[filter, channel,:,:])
#         print("\t\tChannel : Done")
        if W_conv is None:
          W_conv = temp
        else:
          W_conv =  cp.concatenate((W_conv, temp), axis=1) #Ghép theo cột
#       self.W_conv_list.append( W_conv)
      self.W_conv_list[filter] = W_conv

  
  def im2col(self, image): #Chuyển một ma trận 2 chiều (1 channel) về dạng X_conv
    self.o1 = int((self.input.shape[1] - self.filter_size[0])/self.stride) + 1 #Số step dịch kernel theo row
    self.o2 = int((self.input.shape[2] - self.filter_size[1])/self.stride) + 1 #Số step dịch kernel theo column   
    list_conv = None
    for row in range(self.o1):
      row_start = row * self.stride
      row_end = row_start + self.filter_size[0]
      for col in range(self.o2):  
        col_start = col*self.stride
        col_end = col_start + self.filter_size[1]      
        temp = image[row_start:row_end, col_start:col_end].flatten()
        if list_conv is None:
          list_conv = cp.flip(temp,0).reshape(temp.shape[0], 1)
        else:
          list_conv = cp.concatenate((list_conv, cp.flip(temp,0).reshape(temp.shape[0], 1)), axis = 1) 
#     X_conv = list_conv
    return list_conv
    
    
  def kernel2row(self, kernel): #Chuyển một kernel về dạng W_conv
    rot_kernel = cp.flip(kernel, 0)
    rot_kernel = cp.flip(rot_kernel, 1)
    list_w_conv = None
    for row in range(self.o1):
      row_start = row * self.stride
      row_end = row_start + self.filter_size[0]
      for col in range(self.o2):  
        col_start = col*self.stride
        col_end = col_start + self.filter_size[1]  
        temp = cp.zeros((self.input.shape[1], self.input.shape[2]))
        temp[row_start:row_end, col_start:col_end] = rot_kernel
        if list_w_conv is None:
          list_w_conv = temp.flatten()
        else:
          list_w_conv = cp.concatenate((list_w_conv, temp.flatten()), axis = 0)
    W_conv = list_w_conv.reshape((self.o1*self.o2, self.input.shape[1]*self.input.shape[2]))       
    return W_conv
  
  def save_weights(self, filename):
    cp.save('drive/My Drive/Colab Notebooks/Asgmt_ML_CNN/saved_model_cupy/' + filename + '.npy', self.weights)
    
  def load_weights(self, filename):
    self.weights = cp.load('drive/My Drive/Colab Notebooks/Asgmt_ML_CNN/saved_model_cupy/' + filename + '.npy')
  
  

## MaxPooling2D

In [0]:
class MaxPooling2D():
  def __init__(self, pool_size=(2, 2), stride=None):
    self.X = None
    self.pool_indices = list()
    self.pool_size = pool_size  
    if stride is None:
      self.stride = pool_size
    else:
      self.stride = stride
  
  def feed_forward(self, X):
    D, W, H = X.shape    
    W_pool = self.pool_size[0]
    H_pool = self.pool_size[1]   
    W_S = self.stride[0]
    H_S = self.stride[1]   
    WW = 1 + (W - W_pool)//W_S
    HH = 1 + (H - H_pool)//H_S
    DD = D   
    out = cp.zeros((DD, HH, WW))  
    for depth in range(D):
      for row in range(HH):
        for col in range(WW):
          out[depth, row, col] = cp.max(X[depth, row*H_S:row*H_S+H_pool, col*W_S:col*W_S+W_pool])    
    self.cache = X    
    return cp.array(out)
  
  def backward(self, out):
    D, W, H = self.cache.shape
    DD, WW, HH = out.shape   
    W_pool = self.pool_size[0]
    H_pool = self.pool_size[1]   
    W_S = self.stride[0]
    H_S = self.stride[1]   
    dX = cp.zeros(self.cache.shape)
    
    for depth in range(D):
      for row in range(0, HH):
        for col in range(0, WW):
          x_pool = self.cache[depth, row*H_S:row*H_S+H_pool, col*W_S:col*W_S+W_pool]       
          mask = (x_pool == cp.max(x_pool))         
          dX[depth, row*H_S:row*H_S+H_pool, col*W_S:col*W_S+W_pool] = mask* out[depth, row, col]
    return dX

## Dropout

In [0]:
class Dropout():
  def __init__(self, drop_rate = 0.25):
    self.drop_rate = drop_rate
    self.data = None
      
  def drop(self, element): #Mỗi phần tử bị set 0 với xác suất là drop_rate
    rand = cp.random.ranf(1)
    if rand < self.drop_rate:
      return 0
    else:
      return element/(1-self.drop_rate)
    
#   def feed_forward(self, data):
#     vec = cp.vectorize(self.drop) #Map tất cả các phần tử với hàm drop
#     self.data = vec(data)
#     return self.data
  
  def feed_forward(self, data):
    shape = data.shape
    flat = data.flatten()
    for i in range(len(flat)):
      flat[i] = self.drop(flat[i])
    self.data = flat.reshape(shape)
    return self.data

  def backward(self, dY):
    self.data[self.data != 0] = 1
    return cp.multiply(self.data, dY)

## Flatten

In [0]:
class Flatten():
  def __init__(self):
    self.shape = None
    
  def feed_forward(self, data):
    self.shape = data.shape
    return data.flatten()
  
  def backward(self, dY):
    return dY.reshape(self.shape)

## Dense

In [0]:
class Dense():
  def __init__(self, num_units, init_weights = None, init_bias = None, activation = 'relu', learning_rate = 0.01):
    self.num_units = num_units
    self.weights = init_weights
    self.bias = init_bias
    if activation == 'relu':
      self.activation = ReLU()
    elif activation == 'softmax':
      self.activation = Softmax()
    self.lrn_rate = learning_rate

  def feed_forward(self, data):
    self.input = data
    if self.weights is None:
      rand_number = math.sqrt(6/(self.num_units+data.shape[0]))
      self.weights = cp.random.uniform( low = -rand_number, high = rand_number, size = (self.num_units, data.shape[0]))
    if self.bias is None:
      rand_number = math.sqrt(6/(self.num_units+data.shape[0]))
      self.bias  = cp.random.uniform(low = -rand_number, high = rand_number, size = self.num_units)
    A = cp.matmul(self.weights, data) + self.bias
    return self.activation.feed_forward(A)
  
  def backward(self, dY):
    dY = self.activation.backward(dY)
    dW = cp.matmul(dY.reshape(dY.shape[0], 1), self.input.reshape(1, self.input.shape[0]))
    dX =  cp.matmul(self.weights.T, dY)
    self.weights -= dW*self.lrn_rate
    self.bias -= dY*self.lrn_rate
    return dX
  
  def save_weights(self, filename):
    cp.save('drive/My Drive/Colab Notebooks/Asgmt_ML_CNN/saved_model_cupy/' + filename + '.npy', self.weights)
    cp.save('drive/My Drive/Colab Notebooks/Asgmt_ML_CNN/saved_model_cupy/' + filename + '_bias.npy', self.bias)
    
  def load_weights(self, filename):
    self.weights = cp.load('drive/My Drive/Colab Notebooks/Asgmt_ML_CNN/saved_model_cupy/' + filename + '.npy')
    self.bias = cp.load('drive/My Drive/Colab Notebooks/Asgmt_ML_CNN/saved_model_cupy/' + filename + '_bias.npy')
    

# --------------------Testing--------------------

In [0]:
num_class = 10

convol2D_1  = Convolution2D(2, (3, 3), input_shape=(1,28,28), learning_rate = 0.05)
convol2D_2  = Convolution2D(4, (4, 4), input_shape=(2,26,26), learning_rate = 0.05)
max_pooling = MaxPooling2D(pool_size=(2, 2))
dropout_1   = Dropout(0.25)
flatten     = Flatten()
dropout_2   = Dropout(0.5)
dense_1     = Dense(24, activation='relu',learning_rate = 0.05)
dense_2     = Dense(num_class, activation='softmax', learning_rate = 0.05)

In [0]:
# convol2D_1.load_weights('convol2D_1')
# convol2D_2.load_weights('convol2D_2')
# dense_1.load_weights('dense_1')
# dense_2.load_weights('dense_2')

In [0]:
model = Sequential()
model.add(convol2D_1)
model.add(convol2D_2)
model.add(max_pooling)
model.add(dropout_1)
model.add(flatten)
model.add(dropout_2)
model.add(dense_1)
model.add(dense_2)

model.compile(loss = 'cross_entropy')

batch_size = 100
epochs = 1

model.fit(
      X_train_norm[0:100], 
      y_train_norm[0:100],
      batch_size = batch_size,
      num_epoch = epochs
#       validation_data = (X_test_norm[0:30], y_test_norm[0:30])
)

Feedforward through layer:  <class '__main__.Convolution2D'>
Feedforward through layer:  <class '__main__.Convolution2D'>
Feedforward through layer:  <class '__main__.MaxPooling2D'>
Feedforward through layer:  <class '__main__.Dropout'>
Feedforward through layer:  <class '__main__.Flatten'>
Feedforward through layer:  <class '__main__.Dropout'>
Feedforward through layer:  <class '__main__.Dense'>
Feedforward through layer:  <class '__main__.Dense'>
Feedforward through layer:  <class '__main__.Convolution2D'>
Feedforward through layer:  <class '__main__.Convolution2D'>
Feedforward through layer:  <class '__main__.MaxPooling2D'>
Feedforward through layer:  <class '__main__.Dropout'>
Feedforward through layer:  <class '__main__.Flatten'>
Feedforward through layer:  <class '__main__.Dropout'>
Feedforward through layer:  <class '__main__.Dense'>
Feedforward through layer:  <class '__main__.Dense'>
Feedforward through layer:  <class '__main__.Convolution2D'>
Feedforward through layer:  <clas

In [0]:
convol2D_1.save_weights('convol2D_1')
convol2D_2.save_weights('convol2D_2')
dense_1.save_weights('dense_1')
dense_2.save_weights('dense_2')

In [0]:
predicted = model.predict_class(X_test_norm[0:200])
target =  y_test[0:200]

In [0]:
print("Accuracy: ",accuracy_score(predicted, target))
from sklearn import metrics
print(metrics.classification_report(target, predicted))